In [5]:
import pandas as pd
from pathlib import Path
from utils.labeling import get_barrier_events, get_bins
from IPython.display import display

In [6]:
ticker = 'AAPL'
n_cpus = 8
data_path = Path('data/aapl_dollar_bars.h5')


bars = pd.read_hdf(data_path.resolve(), key='key')
bars.drop(['tick_num', 'cum_buy_volume', 'cum_ticks', 'cum_dollar_value'], axis=1, inplace=True)
bars.set_index('date_time', inplace=True)
bars = bars[~bars.index.duplicated()]

display(bars.head())

,open,high,low,close,volume
date_time,,,,,
2019-11-07 16:28:57.758,259.7026,259.7700,259.48,259.66,320759.0
2019-11-07 16:37:26.188,259.6600,259.8500,259.61,259.73,320271.0
2019-11-07 16:46:59.795,259.7300,259.7453,259.50,259.50,321204.0
2019-11-07 16:54:07.176,259.5000,259.6000,259.13,259.18,320711.0
2019-11-07 16:59:24.869,259.1800,259.9500,259.07,259.55,320834.0



## Labeling

In [7]:
pt_sl = [1., 1.]
avg_events_per_day = 10
num_days_avg_vol = 90
min_ret = 0.005
triple_barrier_events = get_barrier_events(
                                         bars=bars,
                                         n_jobs=n_cpus,
                                         avg_events_per_day=avg_events_per_day,
                                         num_days_avg_vol=num_days_avg_vol,
                                         min_ret= min_ret,
                                         pt_sl=pt_sl,
                                         side_prediction=None)

labels = get_bins(triple_barrier_events, bars['close'])
print("Label value counts:")
print(labels.bin.value_counts())
print('labels count: ', labels.shape[0])

Volatility averaged for the last 90 days
1/10 of average daily volatility used as filter_threshold for the cumsum filter


2026-02-07 19:26:46.509817 100.0% apply_pt_sl_on_t1 done after 0.01 minutes. Remaining 0.0 minutes.


Label value counts:
bin
 1    1313
-1     820
 0     196
Name: count, dtype: int64
labels count:  2329


In [8]:
labels

,return,target,bin
2019-11-14 17:21:46.777,0.005366,0.005123,1
2019-11-14 17:41:01.454,0.005965,0.005091,1
2019-11-20 15:30:24.288,-0.005814,0.005314,-1
2019-11-20 15:35:39.169,-0.005542,0.005256,-1
2019-11-20 15:56:41.797,-0.006503,0.005173,-1
...,...,...,...
2020-08-27 13:42:29.145,-0.014276,0.010894,-1
2020-08-27 13:48:54.757,-0.013471,0.010132,-1
2020-08-27 14:08:47.992,-0.007369,0.007340,-1
2020-08-28 13:13:29.189,-0.009113,0.008048,-1
